In [57]:
import pandas as pd
import pulp as p
import pickle

In [58]:
bids = pd.read_csv('./output/bidding_supplier_characteristics_prediction.csv')
bids

,Unnamed: 0,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice,ScaledQuotedPrice,Quality,Timeliness
0,0,1899326,432118,SUPPLIERRPC04,100,85,850,10.0,5.0,9.0
1,1,1899326,432118,SUPPLIERRPC06,100,92,920,3.7,5.0,9.0
2,2,1899326,432118,SUPPLIERRPC05,100,95,950,1.0,9.0,6.0


In [59]:
'''
 Item selected in UI
 Qty is bid quantity
 Price entered in UI
 Call prediction for a item/price to get supplier characteristics
'''
problem = p.LpProblem(name='Supplier_Optimization', sense=p.LpMaximize)
problem

Supplier_Optimization:
MAXIMIZE
None
VARIABLES

In [60]:
# Decision Variables
suppliers = p.LpVariable.dicts("", bids['SupplierId'], cat='Integer', lowBound=0, upBound=100)
suppliers

{'SUPPLIERRPC04': _SUPPLIERRPC04,
 'SUPPLIERRPC06': _SUPPLIERRPC06,
 'SUPPLIERRPC05': _SUPPLIERRPC05}

In [61]:
# Optimization function
priceProblem = p.lpSum(suppliers[bids['SupplierId'][i]] * bids['ScaledQuotedPrice'][i] for i in range(bids['SupplierId'].count()))
priceProblem

10.0*_SUPPLIERRPC04 + 1.0*_SUPPLIERRPC05 + 3.7*_SUPPLIERRPC06 + 0.0

In [62]:
#Call prediction for each supplier to get quality and timeliness
qualityProblem = p.lpSum(suppliers[bids['SupplierId'][i]] * bids['Quality'][i] for i in range(bids['SupplierId'].count()))
print(qualityProblem)


5.0*_SUPPLIERRPC04 + 9.0*_SUPPLIERRPC05 + 5.0*_SUPPLIERRPC06


In [63]:
timelinessProblem =  p.lpSum(suppliers[bids['SupplierId'][i]] * bids['Timeliness'][i] for i in range(bids['SupplierId'].count()))
print(timelinessProblem)

9.0*_SUPPLIERRPC04 + 6.0*_SUPPLIERRPC05 + 9.0*_SUPPLIERRPC06


In [64]:
weightage_price = 0.34
weightage_quality = 0.33
weightage_timeliness = 0.33

problem += (weightage_price * priceProblem) + (weightage_quality * qualityProblem) + (weightage_timeliness * timelinessProblem)
problem

Supplier_Optimization:
MAXIMIZE
8.020000000000001*_SUPPLIERRPC04 + 5.29*_SUPPLIERRPC05 + 5.878*_SUPPLIERRPC06 + 0.0
VARIABLES
0 <= _SUPPLIERRPC04 <= 100 Integer
0 <= _SUPPLIERRPC05 <= 100 Integer
0 <= _SUPPLIERRPC06 <= 100 Integer

In [65]:
# Constraints
# Required Quantity
problem += p.lpSum(suppliers[bids['SupplierId'][i]] for i in range(bids['SupplierId'].count())) == 150

problem

Supplier_Optimization:
MAXIMIZE
8.020000000000001*_SUPPLIERRPC04 + 5.29*_SUPPLIERRPC05 + 5.878*_SUPPLIERRPC06 + 0.0
SUBJECT TO
_C1: _SUPPLIERRPC04 + _SUPPLIERRPC05 + _SUPPLIERRPC06 = 150

VARIABLES
0 <= _SUPPLIERRPC04 <= 100 Integer
0 <= _SUPPLIERRPC05 <= 100 Integer
0 <= _SUPPLIERRPC06 <= 100 Integer

In [66]:
'''
exp_price = 97
exp_quality = 5
exp_timeliness = 6
'''
print("")

In [67]:
'''
if (weightage_price > 0):
    problem += p.lpSum(suppliers[bids['SupplierId'][i]] * bids['ScaledQuotedPrice'][i] for i in range(bids['SupplierId'].count())) <= exp_price
    print(problem)
'''
print("")

In [68]:
'''
if (weightage_quality > 0):
    problem += p.lpSum(suppliers[bids['SupplierId'][i]] * bids['Quality'][i] for i in range(bids['SupplierId'].count())) >= exp_quality
    print(problem)
'''
print("")

In [69]:
'''
if (weightage_timeliness > 0):
    problem += p.lpSum(suppliers[bids['SupplierId'][i]] * bids['Timeliness'][i] for i in range(bids['SupplierId'].count())) >= exp_timeliness
    print(problem)
'''
print("")

In [70]:
# Solve problem
problem.solve()

# Print status
status = p.LpStatus[problem.status]
print("Status:", status)

# Print optimal values of decision variables

supplier = {}
selectedSupplier = ""
for v in problem.variables():
    print(v.name, "=", v.varValue)
    supplier[v.name.replace("_", "")] = v.varValue
    if v.varValue is not None and v.varValue > 0:
        selectedSupplier = v.name
quotedPrice = p.value(problem.objective)
print("Quoted Price : ", quotedPrice)

print("Optimal value for supplier's " + "'" + str(supplier) + "' and the optimized objective function value is " + str(quotedPrice))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/t4/204xh5g57vv3c7mjndmh4y7r0000gn/T/bc5b389d727249ea8e32875e94d0aa1a-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/t4/204xh5g57vv3c7mjndmh4y7r0000gn/T/bc5b389d727249ea8e32875e94d0aa1a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 19 RHS
At line 21 BOUNDS
At line 25 ENDATA
Problem MODEL has 1 rows, 3 columns and 3 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1095.9 - 0.00 seconds
Cgl0004I processed model has 1 rows, 3 columns (3 integer (0 of which binary)) and 3 elements
Cutoff increment increased from 1e-05 to 0.001998
Cbc0012I Integer solution of -1095.9 found by DiveCoefficient after 0 iterations and 0 n

In [71]:
bids

,Unnamed: 0,ItemId,CommodityId,SupplierId,Quantity,Quoted_Price,ExtendedPrice,ScaledQuotedPrice,Quality,Timeliness
0,0,1899326,432118,SUPPLIERRPC04,100,85,850,10.0,5.0,9.0
1,1,1899326,432118,SUPPLIERRPC06,100,92,920,3.7,5.0,9.0
2,2,1899326,432118,SUPPLIERRPC05,100,95,950,1.0,9.0,6.0


In [74]:
row = 0
allottedSuppliers = pd.DataFrame(columns=['Supplier', 'Alloted Quantity', 'Quoted Price', 'Extended Price'])

totalAmount = 0.0
totalAllotedQty = 0.0
for key in supplier:
    supplierKey = key
    alloted_qty = supplier[key]
    if(alloted_qty > 0):
        supplierBidsData = bids.query('SupplierId == "'+key+'"')
        quoted_price = supplierBidsData['Quoted_Price'].unique()[0]
        extendedPrice = alloted_qty * quoted_price
        totalAmount = totalAmount + extendedPrice
        totalAllotedQty = totalAllotedQty + alloted_qty
        print(supplierKey + ", " + str(alloted_qty) + ", " + str(quoted_price) + ", " + str(totalAmount))
        allottedSuppliers.loc[row] = [supplierKey, alloted_qty, quoted_price, extendedPrice]
        row = row + 1
allottedSuppliers.loc[row] = ['Total', totalAllotedQty, '', totalAmount]
print("Total Amount = "+ str(totalAmount))
allottedSuppliers

SUPPLIERRPC04, 100.0, 85, 8500.0
SUPPLIERRPC06, 50.0, 92, 13100.0
Total Amount = 13100.0


,Supplier,Alloted Quantity,Quoted Price,Extended Price
0,SUPPLIERRPC04,100.0,85,8500.0
1,SUPPLIERRPC06,50.0,92,4600.0
2,Total,150.0,,13100.0
